In [ ]:
# Description: Crawls Naver News for articles containing a given keyword (e.g., "수소") from 1990 to 2024 and exports the titles, content, and metadata to Excel.

import re
from requests import get
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta, date
from time import sleep
import random
from openpyxl import Workbook

# ===============================
# 🔹 User Input
# ===============================
keyword = '수소'  # Search keyword
start_date = '1990.01.01'  # Start date
end_date = '2024.12.31'    # End date

# ===============================
# 🔹 Create Excel Workbook
# ===============================
wb = Workbook()
ws = wb.active
ws.title = keyword

# Set column widths
ws.column_dimensions['A'].width = 20
ws.column_dimensions['B'].width = 20
ws.column_dimensions['C'].width = 60
ws.column_dimensions['D'].width = 120
ws.column_dimensions['E'].width = 50
ws.column_dimensions['F'].width = 20  # News outlet

# Write header row
ws.append(['날짜', '키워드', '네이버 뉴스제목', '네이버뉴스 내용', '네이버뉴스 링크', '언론사명'])

# ===============================
# 🔹 HTTP headers for web request
# ===============================
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.naver.com/',
}

# ===============================
# 🔹 Split the date range into 3-month chunks
# ===============================
def get_date_ranges(start_date, end_date, delta=timedelta(days=90)):
    start_dt = datetime.strptime(start_date, '%Y.%m.%d')
    end_dt = datetime.strptime(end_date, '%Y.%m.%d')

    ranges = []
    while start_dt < end_dt:
        temp_end_dt = min(start_dt + delta, end_dt)
        ranges.append((start_dt.strftime('%Y.%m.%d'), temp_end_dt.strftime('%Y.%m.%d')))
        start_dt = temp_end_dt + timedelta(days=1)
    return ranges

# ===============================
# 🔹 Clean illegal characters from text for Excel
# ===============================
def clean_string(value):
    ILLEGAL_CHARACTERS_RE = re.compile(r'[\x00-\x1F\x7F-\x9F\uD800-\uDFFF]')
    if isinstance(value, str):
        return ILLEGAL_CHARACTERS_RE.sub('', value)
    return value

# ===============================
# 🔹 Crawl news links and article content
# ===============================
def fetch_news_by_date_range(keyword, start_date, end_date):
    naver_news_links = []

    # Iterate over search result pages (up to 2000 results)
    for page in range(1, 3001, 10):
        print(f"크롤링 중: 페이지 {page // 10 + 1}")
        url = f"https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{start_date.replace('.', '')}to{end_date.replace('.', '')}&start={page}"

        try:
            response = get(url, headers=headers)
            if response.status_code == 200:
                soup = bs(response.text, 'html.parser')
                news_items = soup.select('a.info')
                if not news_items:
                    break
                for item in news_items:
                    link = item['href']
                    if 'news.naver.com' in link:
                        naver_news_links.append(link)
                sleep(random.uniform(3, 5))
            elif response.status_code == 403:
                print("403 에러 발생, 건너뜁니다.")
                sleep(random.uniform(60, 90))
                continue
            else:
                print(f"Error fetching page: {response.status_code}")
                sleep(random.uniform(60, 90))
                continue
        except Exception as e:
            print(f"예외 발생: {e}")
            sleep(random.uniform(60, 90))
            continue

    # Crawl article content
    print("뉴스 본문 크롤링 시작")
    for idx, link in enumerate(naver_news_links):
        try:
            response = get(link, headers=headers)
            if response.status_code == 200:
                soup = bs(response.text, 'html.parser')
                title = soup.select_one('div.media_end_head_title').text.strip().replace('\n', '')
                content = soup.select_one('article#dic_area').text.strip().replace('\n', '')
                article_date = soup.select_one('span.media_end_head_info_datestamp_time').get_text(strip=True)
                press = soup.select_one('div.media_end_head_top a').text.strip()

                # Remove duplicate press names
                press = re.sub(r'\b(\w+)\b\s*\1', r'\1', press)

                # Append to Excel
                ws.append([
                    clean_string(article_date),
                    clean_string(keyword),
                    clean_string(title),
                    clean_string(content),
                    clean_string(link),
                    clean_string(press)
                ])

                print(f"기사 저장 완료: {title} ({idx + 1}/{len(naver_news_links)})")
                sleep(random.uniform(2, 4))
            else:
                print(f"본문 요청 실패: {response.status_code}")
                sleep(random.uniform(2, 4))
        except Exception as e:
            print(f"본문 크롤링 실패 ({link}): {e}")
            sleep(random.uniform(2, 4))

# ===============================
# 🔹 Run crawler over all date intervals
# ===============================
date_ranges = get_date_ranges(start_date, end_date)
for start, end in date_ranges:
    fetch_news_by_date_range(keyword, start, end)

# ===============================
# 🔹 Save Excel file
# ===============================
today_str = date.today().strftime("%Y%m%d")
filename = f'naver_news_hydrogen_{start_date.replace(".", "")}_{end_date.replace(".", "")}_{today_str}.xlsx'
wb.save(filename)
print(f"✅ Crawling complete. File saved as: {filename}")


페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
